In [1]:
# Configuring
from data_handler.lc_docstore_handler.in_memory_docstore_handler import InMemoryDocstoreHandler
from data_handler.lc_retrieverHandler.retriever_handler import RetrieverHandler
from data_handler.report_handler.pdf_image_report_handler import PdfImageReportHandler
from data_handler.summary_handler.gpt_summary_handler import GPTSummaryHandler
from data_handler.lc_vectorstore_handler.pinecone_vectorstore_handler import PineconeVectorstoreHandler

from dotenv import load_dotenv
import pandas as pd
import os

# Load the .env file
load_dotenv()
os.chdir('..')

report_data_dir = "./data/reports/"
report_name = "report.pdf"
file_list_df = pd.read_csv("./data/reports.csv")

target = "SK텔레콤"
row = file_list_df[file_list_df.company_name == target].iloc[0]

company_name = row["company_name"]
year = row["year"]
url = f"{os.getenv('logblack_url')}{company_name}_{year}.pdf"

# FIXME: TESTING.
url = "https://www.clickdimensions.com/links/TestPDFfile.pdf"

In [2]:
# pdf to components.

components = PdfImageReportHandler(
        company_name=company_name, year=year, report_url=url
        ).splitReport(report_data_dir, report_name)

File downloaded successfully and saved to ./data/reports/SK텔레콤2023/report.pdf


Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved image to ./data/reports/SK텔레콤2023/page_images, 1 files


In [3]:
summarized_components = GPTSummaryHandler().summary(components)

source_url summary generated
tables summary generated
Error on summary generation: Error code: 404 - {'error': {'message': 'The model `GPT-4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


In [ ]:
# USE LangChain from here.
vectorstore_handler = PineconeVectorstoreHandler(
        company_name=company, year=year, embeddingModel='text-embedding-3-large'
        ).getStore()
docstore_handler = InMemoryDocstoreHandler().getStore()

lc_retriever = RetrieverHandler(vectorstore_handler, docstore_handler)
lc_retriever.add(summarized_components)
docstore_handler.export()
